# Chapter 5: Services: enabling clients to discover and talk to pods

## Kubectl Commands
* kubectl get service: Gets and displays info on all existing services
* kubectl exec {name} -- {bash command}: Remotely runs commands inside an existing container of a pod
* gcloud compute firewall-rules create {rule_name} --allow=tcp:{PORT}: Allow external connections to a given node port. 

## Introducing Services
* Reasons that you can't configure each client app to an exact IP address
    * Pods are ephemeral
    * Kubernetes assigns an IP address toa  pod after the pod has been scheduled to a node and before it's started
    * Horizontal scalaing means multiple pods may provide the same service

* Service: 
    * Resource to make a single, constant point of entry to a group of pods providing the same service
        * Clients can open connections to that IP and port
        * Individual pods can still get moved around the cluster at any time!
* Creating services:
    * Connections to a given service are load-balanced across all backing pods
    * Label selectors are used to specify which groups belong to the same set
    * kubectl create svc
* Example Service Workflow:
    * kubectl exec kubia-7nog1 -- curl -s http://10.111.249.153
        * kubia-7nog1 = K8s Node
        * Service IP
    * Curl is executed inside of a container with specified naame
    * Curl sends HTTP GET request
    * Service redirects HTTP connection to a randomly selected pod
    * Pod processes request
    * HTTP resquest is sent back to curl
    * Output of the curl command is sent back to kubectl
* Service Parameters
    * sessionAffinity: ClientIP -> Redirects all requests made by a given IP to the same pod
    * targetPort: Protocol or port number for the pod
    * port: Port number for the service
* Discovering Internal Services
    * How can you get the IP and port of a service prior to creation?
        * Kubernetes initializes some envars that can be useful to view
            * KUBIA_SERVICE_HOST
            * KUBIA_SERVICE_PORT
        * DNS server lookups
            * Domain Name Service: Stores/finds IP addresses for services
            * Each service gets a DNS entry in the internal DNS server
            * Client pods can access the service through its fully qualified domain name (FQDN)

```
# Example Service

apiVersion: v1
kind: Service
metadata: 
  name: external-service
spec:
  ports:
  - name: http
    port: 80
    targetPort: http
  - name: https
    port: 443
    targetPort: https
```

## Introducing Endpoints
* Connecting to services outside the cluster
    * Sometimes we want to have services redirect connections to external IPs
    * Having this allows for having both service load balancing / load discovery while having external connections just like internal services
* Endpoints:
    * List of target IP addresses and ports exposing a service
    * Request -> Service -> Endpoints -> Pods
    * While endpoints are typically attached to services, they are a separate resource
    * Endpoints need to have the same name as the service
    * If you create a service without any target pods then you'll have to also make an endpoints resource!
    * If you want to migrate the service to be inside of k8s, then you can add a selector to the service which allows for automatic managing!

```
# Example Endpoints
apiVersion: v1
kind: Endpoints
metadata:
  name: external-service
subsets:
  - addresses:
    - ip: 11.11.11.11
    - ip: 22.22.22.22
    ports:
    - port: 80
```

## Exposing services to external clients
* Using a NodePort service
    * Principal:
        * Each cluster node opens a port on all of K8s nodes
            * The same port across all of the nodes
        * Traffic is redirected from the port to the underlying service / pods
    * Workflow:
        * Client puts out a request to Port 30123 on a node with a given external IP
        * The node will redirect the traffic to the service
        * The service will redirect the traffic to the Pod via Port 8080. 
    * Before you can open this up, you'll need to configure the firewall rules to allow for external connections to the nodes on taht port
    * Way to get external IPs:
        * kubectl get nodes -o jsonpath='{.items[*].status. ➥ addresses[?(@.type=="ExternalIP")].address}'
    * If you don't specify a nodePort then k8s will pick one automatically
```
# Example NodePort

apiVersion: v1
kind: Service
metadata:
    name: kubia-nodeport
spec:
  type: NodePort
  ports:
  - port: 80
    targetPort: 8080
    nodePort: 30123
  selector:
    app: kubia

```
* Using an external LoadBalancer
    * Similar to NodePort but you connect via the balancer rather than the node's ip:
        * Better in the case that a node goes down!
    * You can get the load balancer IP by doing a simple get on the service
    * No need to mess with firewalls here!
    * Workflow:
        * Client puts out a request to the load balancer with an external IP
        * The load balancer will redirect the traffic to a node on port 32143
        * The node will redirect the traffic to the service
        * The service will redirect the traffic to the Pod via Port 8080. 

```
# Example LoadBalancer
apiVersion: v1
kind: Service
metadata:
  name: kubia-loadbalancer
spec:
  type: LoadBalancer
  ports:
  - port: 80
    targetPort: 8080
    nodePort: 32143
  selector:
app: kubia
```
* Using an Ingress resource
    * LoadBalancers and NodePorts both require their own public IP to work
    * Ingresses only require 1 (even when providing access to many services)
        * The client sends a host and path in the request to show what it wants
    * Ingresses operate at the application layer of the network stack
    * To use an ingress object, you will need to have an ingress controller in the cluster
    * As with LoadBalancers, the IP address is noted in the kubectl get
    * Ingress resources also provisision a LoadBalancer behind the scenes!
    * Workflow:
        * Client does a DNS lookup / connects to kubia.example.com to get the ingress controller IP
        * Client sends a GET request with the header {host: kubia.example.com} to the ingress controller
        * The Ingress controller looks at the header and redirects the traffic to the Ingress -> Service -> Endpoints -> Pod

```
# Example Ingress
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: kubia
spec:
  rules:
  - host: kubia.example.com
    http: paths:
      - path: /kubia
        backend:
          serviceName: kubia
          servicePort: 80
      - path: /bar
        backend:
          serviceName: bar
          servicePort: 80
```

## Examining Readiness Probes
* What happens when you spin up new pods but they aren't ready to take requests?
    * Could need time to load configs / get data
    * Could need time to do some warm up tasks
* Like a liveliness probe, a readiness probe checks to see if a pod is ready
* Types of Readiness Probes:
    * Exec probe:
        * A process is executed.
        * Status is determined by exit code
    * HTTP GET probe: 
        * Sends an HTTP GET request to the container.
        * Status is determined by GET response code
    * TCP Socket Probe: 
        * Opens a TCP connection to a specified port of the container
        * Status is determined by if a connection can be established
* If a pod fails a readiness check then it's removed from the service endpoints
    * Unlike liveness probes, failures don't lead to pod killing / restarting
    * If it later on passes then it's re-added back
* A readiness probe can be defined for each container in the pod
* In the below example, you can toggle readiness based on checking if a certain file exists or not
* If you add a probe, you'll need to delete the old pods as they won't have the check in place

```
# Example Readiness Probe
apiVersion: v1
kind: ReplicationController
...
spec:
  ...
  template:
    ... 
    spec:
      containers:
      - name: kubia
        image: luksa/kubia
        readinessProbe:
          exec:
            command:
            - ls
            - /var/ready

```

## Summary
* Exposes multiple pods that match a certain label selector under a single, stable IP address and port
* Makes services accessible from inside the cluster by default, but allows you to make the service accessible from outside the cluster by setting its type to either NodePort or LoadBalancer
* Enables pods to discover services together with their IP addresses and ports by looking up environment variables
* Allows discovery of and communication with services residing outside the cluster by creating a Service resource without specifying a selector, by creating an associated Endpoints resource instead
* Provides a DNS CNAME alias for external services with the ExternalName ser- vice type
* Exposes multiple HTTP services through a single Ingress (consuming a single IP)
* Uses a pod container’s readiness probe to determine whether a pod should or shouldn’t be included as a service endpoint
* Enables discovery of pod IPs through DNS when you create a headless service